In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import os

from scipy.stats import uniform, randint
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from xgboost import XGBRegressor

from DataTransformer import DataTransformer
from DataProcessingUtils import *
from TrainUtils import *

In [ ]:
data_folder = "data"
train_path = os.path.join(data_folder, "train.csv")

train_df: pd.DataFrame = pd.read_csv(train_path).drop("Id", axis=1)
y = np.log1p(train_df['SalePrice'])
X: pd.DataFrame = train_df.drop(columns='SalePrice', axis=1)

transformer = DataTransformer()
X = transformer.fit_transform(X)
X = X.astype(str)
X = convert_to_numeric(X)

print(f"Is there nan: {np.any(X.isnull())}")

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=98987)

In [ ]:
parameters = {
    "learning_rate": uniform(1e-3,  1e-1),
    "max_depth": randint(1, 100),
    "min_child_weight": randint(1, 15),
    "n_estimators": randint(1, 1000),
    "colsample_bytree": uniform(1e-1,  1)}

grid_model = RandomizedSearchCV(XGBRegressor(), parameters, cv=3, n_jobs=-1)
grid_model.fit(X_train, y_train)
grid_model

In [9]:
gs_model = XGBRegressor(**grid_model.best_params_)
gs_model.fit(X_train, y_train);

In [10]:
evaluate(gs_model, X_train, y_train)
evaluate(gs_model, X_test, y_test)

RMSLE: 0.037523476026545835
RMSLE: 0.15596662611930004
